<a href="https://colab.research.google.com/github/dandamudi-geeta/Quantum-Computing/blob/main/2348512_QC(Lab8).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install qiskit

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 39.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.4/119.4 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 27.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.7/49.7 MB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.5/108.5 kB 3.6 MB/s eta 0:00:00


In [3]:
!pip install qiskit-aer

In [4]:
from qiskit import transpile
from qiskit_aer import AerSimulator

In [6]:
from qiskit import QuantumCircuit
from qiskit.visualization import plot_histogram
import random


**Define Query Gate Implementation**

In [7]:
def deutsch_jozsa_query(n, case):
    """
    Create the query gate (oracle) for the Deutsch-Jozsa problem.

    Args:
        n (int): Number of qubits representing the input (excluding the auxiliary qubit).
        case (str): 'constant' or 'balanced'.

    Returns:
        QuantumCircuit: A circuit implementing the query gate.
    """
    query_gate = QuantumCircuit(n + 1)  # n input qubits + 1 auxiliary qubit

    if case == "constant":
        # For constant functions, the auxiliary qubit's state is unchanged
        if random.choice([0, 1]) == 1:
            query_gate.x(n)  # Apply X gate to auxiliary qubit if f(x) = 1 for all x

    elif case == "balanced":
        # For balanced functions, flip the auxiliary qubit for half the inputs
        for i in range(2**n // 2):  # Create a balanced function
            binary_string = format(i, f'0{n}b')
            for j, bit in enumerate(binary_string):
                if bit == '0':
                    query_gate.x(j)  # Apply X gates to match the input
            query_gate.mcx(list(range(n)), n)  # Apply multi-controlled NOT
            for j, bit in enumerate(binary_string):
                if bit == '0':
                    query_gate.x(j)  # Uncompute X gates

    query_gate = query_gate.to_gate(label="Oracle")
    return query_gate


**Create the Deutsch-Jozsa Circuit**

In [8]:
def deutsch_jozsa_algorithm(n, oracle):
    """
    Create and run the Deutsch-Jozsa algorithm.

    Args:
        n (int): Number of input qubits.
        oracle (QuantumGate): The query gate (oracle) for the problem.

    Returns:
        QuantumCircuit: The Deutsch-Jozsa circuit.
    """
    dj_circuit = QuantumCircuit(n + 1, n)

    # Initialize the auxiliary qubit in |1⟩ state
    dj_circuit.x(n)
    dj_circuit.h(n)

    # Apply Hadamard to all input qubits
    for qubit in range(n):
        dj_circuit.h(qubit)

    # Add the oracle
    dj_circuit.append(oracle, range(n + 1))

    # Apply Hadamard again to all input qubits
    for qubit in range(n):
        dj_circuit.h(qubit)

    # Measure all input qubits
    dj_circuit.measure(range(n), range(n))

    return dj_circuit


**Run the Algorithm**

In [9]:
# Number of input qubits
n = 3  # You can change this value to increase/decrease complexity

In [10]:
# Randomly select a function type
function_type = random.choice(["constant", "balanced"])
print(f"Chosen function type: {function_type}")


Chosen function type: constant


In [11]:
# Create the oracle for the selected function
oracle_gate = deutsch_jozsa_query(n, function_type)

In [12]:
# Create the Deutsch-Jozsa circuit
dj_circuit = deutsch_jozsa_algorithm(n, oracle_gate)

In [13]:
# Display the quantum circuit
print("Deutsch-Jozsa Circuit:")
print(dj_circuit.draw(output='text'))

Deutsch-Jozsa Circuit:
     ┌───┐     ┌─────────┐┌───┐┌─┐      
q_0: ┤ H ├─────┤0        ├┤ H ├┤M├──────
     ├───┤     │         │├───┤└╥┘┌─┐   
q_1: ┤ H ├─────┤1        ├┤ H ├─╫─┤M├───
     ├───┤     │  Oracle │├───┤ ║ └╥┘┌─┐
q_2: ┤ H ├─────┤2        ├┤ H ├─╫──╫─┤M├
     ├───┤┌───┐│         │└───┘ ║  ║ └╥┘
q_3: ┤ X ├┤ H ├┤3        ├──────╫──╫──╫─
     └───┘└───┘└─────────┘      ║  ║  ║ 
c: 3/═══════════════════════════╩══╩══╩═
                                0  1  2 


In [14]:
# Run the circuit using a simulator
simulator = AerSimulator()
compiled_circuit = transpile(dj_circuit, simulator)
result = simulator.run(compiled_circuit).result()

In [15]:
# Get the measurement results
counts = result.get_counts()
print("\nMeasurement Results:")
print(counts)


Measurement Results:
{'000': 1024}


In [16]:
# Analyze the result
if '0' * n in counts:
    print("The function is CONSTANT.")
else:
    print("The function is BALANCED.")


The function is CONSTANT.
